In [1]:
import keras
from keras.models import Sequential
from keras import layers
from keras.datasets import mnist
from keras import backend as K

Using TensorFlow backend.


In [2]:
# 自己定义一种其他形式的激活函数
# 继承layers.Layer
# 先均值求L2的loss
#　分别使用正负进行联结
class Antirectifier(layers.Layer):
    '''
    It can be used in place of a ReLU.
    # Input shape
        2D tensor of shape (samples, n)
    # Output shape
        2D tensor of shape (samples, 2*n)
    '''

    def compute_output_shape(self, input_shape):
        shape = list(input_shape)
        assert len(shape) == 2  # only valid for 2D tensors
        shape[-1] *= 2
        return tuple(shape)

    def call(self, inputs):
        inputs -= K.mean(inputs, axis=1, keepdims=True)
        inputs = K.l2_normalize(inputs, axis=1)
        pos = K.relu(inputs)
        neg = K.relu(-inputs)
        return K.concatenate([pos, neg], axis=1)

In [4]:
# global parameters
batch_size = 128
num_classes = 10
epochs = 10
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
#　注意转换数据类型
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# build the model
model = Sequential()
model.add(layers.Dense(256, input_shape=(784,)))
model.add(Antirectifier())
model.add(layers.Dropout(0.1))
model.add(layers.Dense(256))
model.add(Antirectifier())
model.add(layers.Dropout(0.1))
model.add(layers.Dense(10))
model.add(layers.Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

60000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               200960    
_________________________________________________________________
antirectifier_1 (Antirectifi (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
antirectifier_2 (Antirectifi (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1

In [5]:
# train the model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# next, compare with an equivalent network
# with2x bigger Dense layers and ReLU

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 15s - loss: 0.6032 - acc: 0.9137 - val_loss: 0.1860 - val_acc: 0.9501
Epoch 2/10
60000/60000 [==============================] - 14s - loss: 0.1252 - acc: 0.9666 - val_loss: 0.1060 - val_acc: 0.9673
Epoch 3/10
60000/60000 [==============================] - 14s - loss: 0.0818 - acc: 0.9757 - val_loss: 0.0837 - val_acc: 0.9747
Epoch 4/10
60000/60000 [==============================] - 14s - loss: 0.0632 - acc: 0.9806 - val_loss: 0.0782 - val_acc: 0.9774
Epoch 5/10
60000/60000 [==============================] - 19s - loss: 0.0514 - acc: 0.9844 - val_loss: 0.0741 - val_acc: 0.9781
Epoch 6/10
60000/60000 [==============================] - 15s - loss: 0.0433 - acc: 0.9866 - val_loss: 0.0695 - val_acc: 0.9788
Epoch 7/10
60000/60000 [==============================] - 14s - loss: 0.0367 - acc: 0.9889 - val_loss: 0.0809 - val_acc: 0.9748
Epoch 8/10
60000/60000 [==============================